!time kg download # after configuring Kaggle CLI, this will download the dataset  
!time unzip -q images_training_rev1.zip

In [1]:
!time unzip -q images_training_rev1.zip

8.34user 2.02system 0:10.39elapsed 99%CPU (0avgtext+0avgdata 3944maxresident)k
0inputs+1883320outputs (0major+999minor)pagefaults 0swaps


In [1]:
import pandas as pd
import zipfile


with zipfile.ZipFile('training_solutions_rev1.zip') as zfile:
    with zfile.open('training_solutions_rev1.csv') as f:
        df = pd.read_csv(f)
df.head()

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,...,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.0,0.0,0.0,0.325512
1,100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,...,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.0,0.0,0.0,0.000000
2,100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,...,0.000000,0.094549,0.000000,0.094549,0.189098,0.000000,0.0,0.0,0.0,0.000000
4,100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000


In [2]:
import numpy as np
import os


img_names = df['GalaxyID'].astype('str') + '.jpg'
labels = np.apply_along_axis(np.argmax, 1, df.iloc[:, 1:4].values)

np.random.seed(42)
train_val_test_split = np.random.choice(['train', 'val', 'test'], size=len(df), p=[0.6, 0.2, 0.2])

old_path = 'images_training_rev1/'
new_path = 'imgs/'

for folder_path in ['train', 'val', 'test']:
    for sub_path in set(labels):
        new_dir = new_path + folder_path + '/' + str(sub_path)
        if not os.path.isdir(new_dir):
            print('Making new directory: {}'.format(new_dir))
            os.makedirs(new_dir)

for image_name, dataset, label in zip(img_names, train_val_test_split, labels):
    try:
        os.rename(old_path + image_name, new_path + dataset + '/' + str(label) + '/' + image_name)
    except OSError:
        pass

In [35]:
from utils import *


def get_LeNet_model():
    inp = Input(shape=(3, 224, 224))
    x = Convolution2D(6, 5, 5, activation='relu')(inp)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Convolution2D(16, 5, 5, activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(120, activation='sigmoid')(x) # not a conv layer
    x = Dense(84, activation='sigmoid')(x) # used relu for faster training
    x = Dense(3, activation='softmax')(x)
    model = Model(inp, x)
    model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

train_batches = image.ImageDataGenerator().flow_from_directory(new_path + 'train/', target_size=(224, 224),
            class_mode='categorical', shuffle=True, batch_size=64)
val_batches = image.ImageDataGenerator().flow_from_directory(new_path + 'val/', target_size=(224, 224),
            class_mode='categorical', shuffle=False, batch_size=64)

Found 36964 images belonging to 3 classes.
Found 12383 images belonging to 3 classes.


In [36]:
%%time

LeNet = get_LeNet_model()
LeNet.optimizer.lr = 0.1
LeNet.fit_generator(train_batches, train_batches.N, nb_epoch=1, 
                    validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
36964/36964 [==============================] - 547s - loss: 0.7427 - acc: 0.5246 - val_loss: 0.7118 - val_acc: 0.5621
CPU times: user 10min 2s, sys: 48.8 s, total: 10min 51s
Wall time: 9min 11s


In [17]:
%%time
from glob import glob

val_batches = image.ImageDataGenerator().flow_from_directory(new_path + 'val/', target_size=(224, 224),
            class_mode='categorical', shuffle=False, batch_size=64)

num_val_pictures = len(glob('imgs/val/*/*'))
pics, pic_labels = [], []


while len(pics) < num_val_pictures:
    x, y = val_batches.next()
    pics.extend(x)
    pic_labels.extend(y)

Found 12383 images belonging to 3 classes.


In [31]:
val_batches = image.ImageDataGenerator().flow_from_directory(new_path + 'val/', target_size=(224, 224),
            class_mode='categorical', shuffle=False, batch_size=64)

predictions = LeNet.predict_generator(val_batches, val_samples=val_batches.N)

Found 12383 images belonging to 3 classes.


In [33]:
Counter(np.apply_along_axis(np.argmax, 1, predictions))
# something wrong. always predicting class 0

Counter({0: 12383})

In [34]:
%%time
from collections import Counter

print(np.array(pics).shape)
print(Counter([np.argmax(tup) for tup in pic_labels]))

(12383, 3, 224, 224)
Counter({1: 6961, 0: 5410, 2: 12})
CPU times: user 2.25 s, sys: 3.25 s, total: 5.5 s
Wall time: 5.5 s


In [6]:
%%time
LeNet.optimizer.lr = 0.01
LeNet.fit_generator(train_batches, train_batches.N, nb_epoch=2, 
                    validation_data=val_batches, nb_val_samples=val_batches.N)
LeNet.optimizer.lr = 0.001
LeNet.fit_generator(train_batches, train_batches.N, nb_epoch=2, 
                    validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/2
36964/36964 [==============================] - 553s - loss: 9.1457 - acc: 0.4326 - val_loss: 9.0763 - val_acc: 0.4369
Epoch 2/2
36964/36964 [==============================] - 547s - loss: 9.1575 - acc: 0.4319 - val_loss: 9.0763 - val_acc: 0.4369
Epoch 1/2
36964/36964 [==============================] - 555s - loss: 9.1357 - acc: 0.4332 - val_loss: 9.0763 - val_acc: 0.4369
Epoch 2/2
36964/36964 [==============================] - 545s - loss: 9.1444 - acc: 0.4327 - val_loss: 9.0763 - val_acc: 0.4369


In [7]:
LeNet.optimizer.lr = 0.1
LeNet.fit_generator(train_batches, train_batches.N, nb_epoch=2, 
                    validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/2
36964/36964 [==============================] - 549s - loss: 9.1601 - acc: 0.4317 - val_loss: 9.0763 - val_acc: 0.4369
Epoch 2/2
36964/36964 [==============================] - 544s - loss: 9.1561 - acc: 0.4319 - val_loss: 9.0763 - val_acc: 0.4369


In [ ]:
!rm -rf images_training_rev1/
!rm -rf imgs

In [10]:
"""
from vgg16 import Vgg16
import os

path = '../courses/deeplearning1/nbs/data/distracted_driver/imgs/' # change for respective data set
model_path = '../courses/deeplearning1/nbs/data/distracted_driver/models/' # change for respective data set
batch_size = 64

if not os.path.isdir(model_path): # make validation folder
    print('Making new directory: {}'.format(model_path))
    os.mkdir(model_path)

vgg = Vgg16() # imagenet weights already loaded
# during training, shuffle should be set to true. Or else, it will likely receive training on the same class consecutively
train_batches = vgg.get_batches(path + 'train/', batch_size=batch_size, shuffle=True)
val_batches = vgg.get_batches(path + 'val/', batch_size=batch_size * 2, shuffle=False)
vgg.finetune(train_batches) # optimizer is Adam
"""

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.


In [4]:
from vgg16 import Vgg16

new_path = 'imgs/'
batch_size = 64


vgg = Vgg16() # imagenet weights already loaded
# during training, shuffle should be set to true. Or else, it will likely receive training on the same class consecutively
train_batches = vgg.get_batches(new_path + 'train/', batch_size=batch_size, shuffle=True)
val_batches = vgg.get_batches(new_path + 'val/', batch_size=batch_size * 2, shuffle=False)
vgg.finetune(train_batches) # optimizer is Adam

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


Found 36964 images belonging to 3 classes.
Found 12383 images belonging to 3 classes.


In [5]:
vgg.get_batches??

In [ ]:
vgg.fit(train_batches, val_batches, nb_epoch=1)

Epoch 1/0
 3200/36964 [=>............................] - ETA: 1011s - loss: 0.7715 - acc: 0.6669

In [ ]:
# what does vgg.finetune do? how to do a mixiterator type of thing to get probabilistic inputs